## [Pytorch Forecasting | Temporal Fusion Transformer](https://www.kaggle.com/code/crustacean/pytorch-forecasting-temporal-fusion-transformer)

In [ ]:
import pandas as pd

train = pd.read_csv('../../data/sales_train.csv')
test = pd.read_csv('../../data/sales_test.csv')
solution = pd.read_csv('../../data/solution.csv')
inv = pd.read_csv('../../data/inventory.csv')
cle = pd.read_csv('../../data/calendar.csv')
test_weights = pd.read_csv('../../data/test_weights.csv')
train = train.merge(inv,on=['warehouse','unique_id'],how='left')
train['unique_id_and_name'] = train['unique_id'].astype(str) + '_' + train['name']
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])